In [1]:
import pandahouse as ph
import pandas as pd
import numpy as np
import hashlib
from scipy import stats
from scipy.stats import ttest_ind
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import pylab

In [2]:
connection = {'host': 'https://clickhouse.lab.karpov.courses',
                      'database':'simulator_20230420',
                      'user':'student', 
                      'password':'dpo_python_2020'
                     }

In [3]:
q = """
SELECT exp_group, 
    user_id,
    sum(action = 'like') as likes,
    sum(action = 'view') as views,
    likes/views as ctr
FROM {db}.feed_actions 
WHERE toDate(time) between '2023-04-01' and '2023-04-07'
    and exp_group in (0,3)
GROUP BY exp_group, user_id
"""

df = ph.read_clickhouse(q, connection=connection)

df.groupby('exp_group').count()

,user_id,likes,views,ctr
exp_group,,,,
0,9999,9999,9999,9999
3,10049,10049,10049,10049


In [6]:
ctr_control = df[df.exp_group == 0].likes.sum() / df[df.exp_group == 0].views.sum() # 𝐶𝑇𝑅𝑐𝑜𝑛𝑡𝑟𝑜𝑙=𝑠𝑢𝑚(𝑙𝑖𝑘𝑒𝑠)/𝑠𝑢𝑚(𝑣𝑖𝑒𝑤𝑠)

In [7]:
ctr_control

0.20823612262916305

In [8]:
df['linearized_likes'] = df['likes'] - ctr_control * df['views'] #𝑙𝑖𝑛𝑒𝑎𝑟𝑖𝑧𝑒𝑑_𝑙𝑖𝑘𝑒𝑠 =𝑙 𝑖𝑘𝑒𝑠−𝐶𝑇𝑅𝑐𝑜𝑛𝑡𝑟𝑜𝑙∗𝑣𝑖𝑒𝑤𝑠

In [ ]:
df

,exp_group,user_id,likes,views,ctr,linearized_likes
0,3,115383,9,30,0.300000,2.752916
1,3,123580,13,48,0.270833,3.004666
2,0,4944,8,41,0.195122,-0.537681
3,0,4504,5,15,0.333333,1.876458
4,0,121508,18,88,0.204545,-0.324779
...,...,...,...,...,...,...
20043,3,13781,10,27,0.370370,4.377625
20044,3,130651,18,98,0.183673,-2.407140
20045,0,3681,7,55,0.127273,-4.452987
20046,3,122008,5,41,0.121951,-3.537681


In [10]:
ttest_ind(df[df.exp_group == 0].ctr, df[df.exp_group == 3].ctr, equal_var = False)

Ttest_indResult(statistic=-13.896870721904069, pvalue=1.055849414662529e-43)

In [11]:
ttest_ind(df[df.exp_group == 0].linearized_likes, df[df.exp_group == 3].linearized_likes, equal_var = False)

Ttest_indResult(statistic=-15.21499546090383, pvalue=5.4914249479687664e-52)

Между стандартным ctr двух групп и линеаризованных  кликах есть разница. На линеаризованных кликах pvalue намного меньше, метрика стала чувствительнее.

Возьмем вторую группу

In [13]:
q = """
SELECT exp_group, 
    user_id,
    sum(action = 'like') as likes,
    sum(action = 'view') as views,
    likes/views as ctr
FROM {db}.feed_actions 
WHERE toDate(time) between '2023-04-01' and '2023-04-07'
    and exp_group in (1,2)
GROUP BY exp_group, user_id
"""

df = ph.read_clickhouse(q, connection=connection)

df.groupby('exp_group').count()

,user_id,likes,views,ctr
exp_group,,,,
1,10079,10079,10079,10079
2,9952,9952,9952,9952


In [14]:
ctr_control = df[df.exp_group == 1].likes.sum() / df[df.exp_group == 1].views.sum() # 𝐶𝑇𝑅𝑐𝑜𝑛𝑡𝑟𝑜𝑙=𝑠𝑢𝑚(𝑙𝑖𝑘𝑒𝑠)/𝑠𝑢𝑚(𝑣𝑖𝑒𝑤𝑠)

In [15]:
ctr_control

0.20802680490126244

In [16]:
df['linearized_likes'] = df['likes'] - ctr_control * df['views'] # 𝑙𝑖𝑛𝑒𝑎𝑟𝑖𝑧𝑒𝑑_𝑙𝑖𝑘𝑒𝑠=𝑙𝑖𝑘𝑒𝑠−𝐶𝑇𝑅𝑐𝑜𝑛𝑡𝑟𝑜𝑙∗𝑣𝑖𝑒𝑤𝑠

In [17]:
df

,exp_group,user_id,likes,views,ctr,linearized_likes
0,1,109963,3,15,0.200000,-0.120402
1,1,26117,32,141,0.226950,2.668221
2,1,138232,18,73,0.246575,2.814043
3,1,26295,33,122,0.270492,7.620730
4,1,18392,7,32,0.218750,0.343142
...,...,...,...,...,...,...
20026,1,132567,45,296,0.152027,-16.575934
20027,1,30872,11,59,0.186441,-1.273581
20028,2,25383,42,127,0.330709,15.580596
20029,2,139020,24,114,0.210526,0.284944


In [18]:
ttest_ind(df[df.exp_group == 1].ctr, df[df.exp_group == 2].ctr, equal_var = False)

Ttest_indResult(statistic=0.7094392041270485, pvalue=0.4780623130874935)

In [19]:
ttest_ind(df[df.exp_group == 1].linearized_likes, df[df.exp_group == 2].linearized_likes, equal_var = False)

Ttest_indResult(statistic=6.122579994775973, pvalue=9.439432187037712e-10)

Здесь мы также видим сильное снижение pvalue на линеаризованных лайках.

## ВЫВОД
#### Метод очень хорошо детектит изменения, даже очень маленькие.